In [2]:
# importing packages
import pandas as pd
import re
import os
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# import ChromeDriverManager
from time import sleep
from time import time
start_time = time()
from selenium.webdriver.chrome.service import Service
from warnings import warn
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# from fake_useragent import UserAgent

import lxml
import requests
import urllib.request
from urllib.request import urlopen
from urllib.request import urlparse
from urllib.error import HTTPError
import numpy as np

from dateutil.relativedelta import relativedelta
# import timeago, datetime

import matplotlib.pyplot as plt
import seaborn as sns
import random
import locale
from tabulate import tabulate
from datetime import timedelta

ModuleNotFoundError: No module named 'tabulate'

In [ ]:
# replace variables here.
url_template = 'https://www.opcionempleo.com.mx/empleo-energia.html?p={}&sort=date'


In [3]:

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--incognito')
options.add_argument("start-maximized");
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")


In [4]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome("C:/Users/jparr/OneDrive - DAI/Mexico_FCO_Energy/chromedriver/chromedriver.exe", options = options)
driver.get(url_template)
sleep(7)
action = ActionChains(driver)
driver.set_window_size(1024, 600)
driver.maximize_window()

In [5]:
no_of_jobs = 1200

In [6]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('div', {'id': 'search-content'})

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 17 jobs.


In [7]:
opcionempleo_df = pd.DataFrame(columns=["Title","Company", "Location", "Salary", "Date", "Synopsis", "Link"])
max_results = 57
i = 1

for start in range(0, max_results, 1):
    # get results from request above
    url = url_template.format(start)
    html = driver.get(url)
    sleep(1)
    pageSource = driver.page_source
    lxml_soup = BeautifulSoup(pageSource, 'lxml')
    job_container = lxml_soup.find('div', {'id': 'search-content'})
    sleep(5)
    for each in job_container.find_all("article", attrs={"class":"job"}):
        try: 
            title = each.find("h2").find("a").get("title").replace('\n', '')
        except:
            title = 'None'
        try:
            company = each.find("p", class_="company").get_text().replace('\n', '')
        except:
            company = 'None'
        try:
            location = each.find("ul", {"class":'location'}).get_text().replace('\n', '').strip()
        except:
            location = 'None'
        try:
            date =  each.find('span', {"class":'badge'}).text.replace('\n', '')
        except:
            date = 'None'
        try:
            salary = each.find('ul', {"class":'salary'}).get_text().replace('\n', '').strip()
        except:
            salary = 'None'
        try:
            link = 'http://www.opcionempleo.com.mx' + each.find("h2").find("a").get("href").replace('\n', '')
        except:
            link = "None"
        try:
            synopsis = each.find("div", class_="desc").getText().replace('\n', '')
        except:
            synopsis = 'None'
        opcionempleo_df = opcionempleo_df.append({'Title':title, 'Company':company, 'Location':location, 'Salary':salary, 'Date':date, 'Synopsis':synopsis, 'Link':link}, ignore_index=True)

opcionempleo_df = opcionempleo_df.drop_duplicates()
opcionempleo_df = opcionempleo_df[opcionempleo_df.Title != "None"]

descriptions = []

opemp = pd.DataFrame(columns=["Description", "Link"])



In [8]:
opcionempleo_df

,Title,Company,Location,Salary,Date,Synopsis,Link
0,Plant Facilities Engineer Specialist,Stellantis,"Saltillo, Coah.",None,Hace 13 horas,Responsable de la administración eficiente...,http://www.opcionempleo.com.mx/jobad/mx89278db...
1,Mecánico electricista,PPG,"Tepotzotlán, Méx.",None,Hace 14 horas,"Como Mecánico Electricista, desarrollarás ...",http://www.opcionempleo.com.mx/jobad/mx02a9457...
2,FullStack Developer Jr,Bosch,México,None,Hace 14 horas,Company Description Do you want benefici...,http://www.opcionempleo.com.mx/jobad/mxafc9a3d...
3,Vendedores de piso de Temporada,H&M,"Zapopan, Jal.",None,Hace 15 horas,Descripción del empleo VENDEDOR DE PISO ...,http://www.opcionempleo.com.mx/jobad/mxb111512...
4,Ayudante General,PPG,"Reynosa, Tamps.",None,Hace 16 horas,"Como Ayudante General, serás responsable d...",http://www.opcionempleo.com.mx/jobad/mxe62d685...
...,...,...,...,...,...,...,...
1135,Retail Customer Advisor - SIS Monterrey,Ermenegildo Zegna,Nuevo Leon,None,Hace 1 mes,ASESOR DE VENTAS – ZEGNA SIS MONTERREY E...,http://www.opcionempleo.com.mx/jobad/mx1812aa5...
1136,Devops,TRS Staffing Solutions,México,None,Hace 1 mes,You are subscribed to our push notificatio...,http://www.opcionempleo.com.mx/jobad/mxa1497c2...
1137,AZURE Data Engineer,TRS Staffing Solutions,México,None,Hace 1 mes,You are subscribed to our push notificatio...,http://www.opcionempleo.com.mx/jobad/mx51055fb...
1138,Field Specialist II - Production Operations,Baker Hughes,Veracruz,None,Hace 1 mes,Field Specialist-Productions Operations ...,http://www.opcionempleo.com.mx/jobad/mxdc3aa4b...


In [9]:
for link in opcionempleo_df.iterrows():
    url = link[1]['Link']
    html = driver.get(url)
    pageSource = driver.page_source
    soup = BeautifulSoup(pageSource, 'lxml')
    sleep(5)
    try:
        description = soup.find('section', {"class":"content"}).get_text().replace('\n', '')
    except:
        description = 'None'
    descriptions.append(description)
    opemp = opemp.append({'Description':description, 'Link':url}, ignore_index = True)
    sleep(3)




In [10]:
opemp

,Description,Link
0,Responsable de la administración eficien...,http://www.opcionempleo.com.mx/jobad/mx89278db...
1,"Como Mecánico Electricista, desarrollará...",http://www.opcionempleo.com.mx/jobad/mx02a9457...
2,Company Description Do you want beneficial te...,http://www.opcionempleo.com.mx/jobad/mxafc9a3d...
3,Descripción del empleo VENDEDOR DE PISO ¿Est...,http://www.opcionempleo.com.mx/jobad/mxb111512...
4,"Como Ayudante General, serás responsable...",http://www.opcionempleo.com.mx/jobad/mxe62d685...
...,...,...
1115,ASESOR DE VENTAS – ZEGNA SIS MONTERREY ...,http://www.opcionempleo.com.mx/jobad/mx1812aa5...
1116,You are subscribed to our push notificat...,http://www.opcionempleo.com.mx/jobad/mxa1497c2...
1117,You are subscribed to our push notificat...,http://www.opcionempleo.com.mx/jobad/mx51055fb...
1118,Field Specialist-Productions Operations ...,http://www.opcionempleo.com.mx/jobad/mxdc3aa4b...


In [11]:

# opcionempleo_df['Description'] = descriptions

# split location into two
opcionempleo_df[['Municipality', 'State']] = opcionempleo_df['Location'].str.split(pat = ",", n = 1, expand=True)

# drop duplicates    
opcionempleo_df = opcionempleo_df.drop_duplicates()

# join
opcionempleo_df = opcionempleo_df.join(opemp.set_index('Link'), on='Link')

# save temp file
# opcionempleo_df.to_excel('opcionempleo_df_july_23(1).xlsx', sheet_name = 'Sheet 1')


### clean data - opcion empleo

#### location ####

opcionempleo_df.loc[opcionempleo_df['State'].isnull() & opcionempleo_df['Municipality'].notnull(), ['Municipality', 'State']] = opcionempleo_df.loc[opcionempleo_df['State'].isnull() & opcionempleo_df['Municipality'].notnull(), ['State', 'Municipality']].values


opcionempleo_df['Municipality'] = opcionempleo_df['Municipality'].str.replace('Ciudad de México', 'None')

opcionempleo_df.State = opcionempleo_df.State.str.strip()
opcionempleo_df.Municipality = opcionempleo_df.Municipality.str.strip()

opcionempleo_df = opcionempleo_df.replace({'State' : {'Ciudad de México' : 'Ciudad de México (Distrito Federal)',
                              'N.L.' : 'Nuevo León', 
                              'Hgo.' : 'Hidalgo', 
                              'Chis.' : 'Chiapas', 
                              'Jal.' : 'Jalisco', 'Dgo.': 'Durango',
                              'CDMX' : 'Ciudad de México (Distrito Federal)' ,
                              'D.F.' : 'Ciudad de México (Distrito Federal)' ,
                              'Tlax.' : 'Tlaxcala', 
                              'Chih.' : 'Chihuahua', 
                              'B.C.' : 'Baja California', 
                              'Qro.' : 'Querétaro', 
                              'S.L.P.': 'San Luis Potosí', 
                              'Gto.' : 'Guanajuato',
                              'Méx.' : 'Estado de México',
                              'Coah.' : 'Coahuila', 
                              'Tab.' : 'Tabasco', 
                              'Pue.' : 'Puebla (de los Angeles)',
                              'Mor.' : 'Morelos', 
                              'Tamps.': 'Tamaulipas', 
                              'QRoo.' : 'Quintana Roo',
                              'Ver.' : 'Veracruz', 
                              'Camp.' : 'Campeche', 
                              'Ags.' : 'Aguascalientes',
                              'Nay.' : 'Nayarit', 
                              'Mich.' : 'Michoacán', 
                              'Col.' : 'Colima', 
                              'Sin.' : 'Sinaloa',
                              'Son.' : 'Sonora', 
                              'Yuc.' : 'Yucatán', 
                              'Oax.' : 'Oaxaca', 
                              'B.C.S.': 'Baja California Sur', 
                              'Zac.' : 'Zacatecas',
                              'México': 'None'}})


opcionempleo_df.loc[opcionempleo_df['State'] == 'Ciudad de México (Distrito Federal)', ['State', 'Municipality']]


opcionempleo_df.loc[opcionempleo_df['State'] == 'Roma', 'Municipality'] = 'Roma'

opcionempleo_df[['State']] = opcionempleo_df[['State']].replace(['Roma', 'Cuenca'], ['Ciudad de México (Distrito Federal)', 'None'])

opcionempleo_df.loc[opcionempleo_df['State'].isin(['Nuevo León','Hidalgo','Chiapas',
                               'Jalisco','Durango','Ciudad de México (Distrito Federal)',
                               'Tlaxcala','Chihuahua','Baja California','Querétaro',
                               'San Luis Potosí','Guanajuato','Estado de México',
                               'Coahuila','Tabasco','Puebla (de los Angeles)',
                               'Morelos','Tamaulipas','Quintana Roo', 'Veracruz',
                               'Campeche','Aguascalientes','Nayarit','Michoacán',
                               'Colima','Sinaloa','Sonora','Yucatán','Oaxaca',
                               'Baja California Sur','Zacatecas','None']) == False, 'State'] = 'None'



#### date ####
def opemp_get_past_date(str_days_ago):
    TODAY = datetime.date.today()
    splitted = str_days_ago.split()
    if len(splitted) == 3 and splitted[1].lower() in ['minutos']:
        return str(TODAY.isoformat())
    elif len(splitted) == 3 and splitted[2].lower() in ['horas']:
        return str(TODAY.isoformat())
    elif len(splitted) == 1 and splitted[1].lower() in ['ahora']:
        return str(TODAY.isoformat())
    elif len(splitted) == 3 and splitted[2].lower() in ['días', 'día']:
        date = TODAY - relativedelta(days=int(splitted[1]))
        return str(date.isoformat())
    elif len(splitted) == 3 and splitted[2].lower() in ['días']:
        date = TODAY - relativedelta(days=30)
        return str(date.isoformat())
    elif len(splitted) == 3 and splitted[2].lower() in ['mes']:
        date = TODAY - relativedelta(months=12)
        return str(date.isoformat())
    elif len(splitted) == 3 and splitted[2].lower() in ['meses']:
        date = TODAY - relativedelta(months =int(splitted[1]))
        return str(date.isoformat())
    else:
        return "Wrong Argument format"


op_dates = []
for op_date in opcionempleo_df['Date']:
    op_date = opemp_get_past_date(op_date)
    op_dates.append(op_date)
    

opcionempleo_df['Date'] = op_dates


### salary ###


opcionempleo_df['Salary'] = opcionempleo_df['Salary'].replace('\\n', '', regex= True)

opcionempleo_df['Frequency'] = opcionempleo_df['Salary'].str.rsplit(' ', 1).str[-1]


opcionempleo_df['Salary'] = opcionempleo_df.apply(lambda row : row['Salary'].replace(str(row['Frequency']), ''), axis=1)

# opcionempleo_df[(~opcionempleo_df.Salary.str.contains('por hora')) & (~opcionempleo_df.Salary.str.contains('al mes'))
#                    & (~opcionempleo_df.Salary.str.contains('por semana')) & (~opcionempleo_df.Salary.str.contains('al día'))]

opcionempleo_df.Salary = opcionempleo_df.Salary.str.replace('.', '').str.replace('$', '')
opcionempleo_df.Salary = opcionempleo_df.Salary.str.replace('a', '-').str.strip()

op_new_salaries = []
for i in opcionempleo_df.Salary:
    a = i.split(' - ')
    if len(a) == 2:
        op_new_salaries.append(np.mean([float(b) for b in a if a != 'None']))
    else:
        op_new_salaries.append(a[0])
        
opcionempleo_df.Salary = op_new_salaries

opcionempleo_df['Salary'] = pd.to_numeric(opcionempleo_df['Salary'], errors = "coerce")
opcionempleo_df.Salary = opcionempleo_df.Salary.astype(float)


opcionempleo_df['Salary'] = np.where(opcionempleo_df['Frequency']=='año', opcionempleo_df['Salary'].div(12), opcionempleo_df['Salary'])
opcionempleo_df['Salary'] = np.where(opcionempleo_df['Frequency']=='semana', opcionempleo_df['Salary']*4, opcionempleo_df['Salary'])
opcionempleo_df['Salary'] = np.where(opcionempleo_df['Frequency']=='día', opcionempleo_df['Salary']*30, opcionempleo_df['Salary'])

opcionempleo_df = opcionempleo_df.drop(columns = ['Frequency'])

# clean company

opcionempleo_df['Company'] = opcionempleo_df['Company'].str.strip()

# remove following
remove = ['Michael Page', 'Page Personnel', 'Job Fit', 'Working Mexico Headhunter', 'Page Personnel México',
        'Confidential', 'Confidencial', 'To view company name. See instructions below', 'Colocandote',
        'Working Mexico Headhunter', 'RemoteJobsWW', 'PageExecutive', 'Bumeran', 'Randstad', 'Adecco',
        'Cornerjobmx', 'Energyjobline', 'Computrabajo.Com', 'Confidencial', 'ListoPro',
        'Valle de Santiago', 'Texcoco', 'Benito Juárez', 'Monterrey', 'Coatzacoalcos', 'Silao', 'Mérida', 'San Martín de las Pirámides',
       'San Luis Potosí', 'Salvatierra', 'Miguel Hidalgo', 'Ecatepec de Morelos', 'Naucalpan de Juárez', 'Cuauhtémoc',
       'Veracruz', 'Salamanca', 'Juventino Rosas', 'Manzanillo', 'Oaxaca', 'Guadalajara', 'Puerto Vallarta', 'Iztapalapa', 'Guanajuato',
       'Minatitlán', 'Santa Cruz de Juventino Rosas', 'Tehuacán', 'Yautepec', 'Huixquilucan', 'Mexicali', 'Tlaquepaque', 'Morelia',
       'Guadalupe', 'Apodaca', 'Metepec', 'Lerma', 'Cuajimalpa de Morelos', 'Gustavo A. Madero', 'Toluca',
       'Tuxtla Gutiérrez', 'Zapopan', 'Atizapán de Zaragoza', 'Puebla',
       'Tlalnepantla de Baz', 'Gómez Palacio', 'Tlalpan', 'Cordoba', 'Nezahualcóyotl', 'Pachuca de Soto', 'Alvaro Obregón', 'Apizaco',
       'Lampazos de Naranjo', 'Tamazunchale', 'Santa Catarina',
       'Tlaxcala', 'Querétaro', 'Hermosillo', 'Chihuahua', 'Jiutepec', 'Paraíso', 'Torreon', 'Amatlán de los Reyes', 'Ciudad Juárez',
       'San Andrés Cholula', 'Azcapotzalco', 'Jocotitlán',
       'Aguascalientes', 'Roma', 'Zacoalco de Torres', 'Reynosa', 'Cuautitlán de García Barragán', 'Tijuana', 'Corregidora',
       'Cozumel', 'Isla Mujeres', 'Emiliano Zapata', 'Xalapa',
       'San Juan Del Rio', 'Zacatecas', 'Fresnillo', 'Los Cabos', 'Ensenada', 'Huejotzingo', 'Saltillo', 'Ixtlahuaca', 'Ahuazotepec', 'León', 'Tetla', 
       'Calera', 'Casas Grandes', 'Chalco',
       'Zapotlanejo', 'Ocoyoacac', 'Tepotzotlán', 'Otzolotepec', 'Irapuato', 'Río Bravo', 'San Martín Texmelucan',
       'Cuautitlán Izcalli', 'Macuspana', 'Temozón', 'Cárdenas', 'Teapa', 'San Juan Bautista Tuxtepec', 'Acayucan', 'Ciudad del Carmen',
       'Tecate', 'Tepic', 'Arandas', 'Cancún', 'El Espinal', 'Madero', 'Nuevo León','Hidalgo','Chiapas',
       'Jalisco','Durango','Ciudad de México (Distrito Federal)',
       'Tlaxcala','Chihuahua','Baja California','Querétaro','San Luis Potosí','Guanajuato','Estado de México',
       'Coahuila','Tabasco','Puebla (de los Angeles)','Morelos','Tamaulipas','Quintana Roo', 'Veracruz',
       'Campeche','Aguascalientes','Nayarit','Michoacán','Colima','Sinaloa','Sonora','Yucatán','Oaxaca',
       'Baja California Sur','Zacatecas']


opcionempleo_df.loc[opcionempleo_df['Company'].isin(remove) == True, 'Company'] = "None"


# finalize columns 
# opcionempleo_df.columns


opcionempleo_df['Level'] = 'None'
opcionempleo_df['Sector'] = 'None'
opcionempleo_df['Type'] = 'None'
opcionempleo_df['Function'] = 'None'
opcionempleo_df['Source'] = 'OpcionEmpleo'


opcionempleo_df.columns = opcionempleo_df.columns.str.lower()


# drop duplicates based on link column (same link = same job)
opcionempleo_df = opcionempleo_df.drop_duplicates(subset='link', keep="first")

opcionempleo_df.to_excel('opcionempleo_df_clean.xlsx', sheet_name = 'Sheet 1', index = False)

In [12]:

# upload to sql

# test posgresql database

import psycopg2
import psycopg2.extras as extras
import os
from io import StringIO
import sqlalchemy


# Here you want to change your database, username & password according to your own values
param_dic = {
    "host"      : "mexico-fco-db.cmjm2q5ghgua.us-east-1.rds.amazonaws.com",
    "database"  : "postgres",
    "user"      : "mexicoFCO",
    "password"  : "HC2g8Q7^taOq"
}



connection = psycopg2.connect(host = "mexico-fco-db.cmjm2q5ghgua.us-east-1.rds.amazonaws.com",
                 database = "postgres",
                 user = "mexicoFCO",
                 password = "HC2g8Q7^taOq")

cursor = connection.cursor()


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)


def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s) ON CONFLICT (link) DO UPDATE SET link = excluded.link;" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()
    

execute_many(conn = conn, df = opcionempleo_df, table = 'jobs_dbase')
# WORKS!

Connecting to the PostgreSQL database...
Connection successful
execute_many() done


In [12]:
opcionempleo_df

,title,company,location,salary,date,synopsis,link,municipality,state,description,level,sector,type,function,source
0,Plant Facilities Engineer Specialist,Stellantis,"Saltillo, Coah.",NaN,2021-09-30,Responsable de la administración eficiente...,http://www.opcionempleo.com.mx/jobad/mx89278db...,Saltillo,Coahuila,Responsable de la administración eficien...,None,None,None,None,OpcionEmpleo
1,Mecánico electricista,PPG,"Tepotzotlán, Méx.",NaN,2021-09-30,"Como Mecánico Electricista, desarrollarás ...",http://www.opcionempleo.com.mx/jobad/mx02a9457...,Tepotzotlán,Estado de México,"Como Mecánico Electricista, desarrollará...",None,None,None,None,OpcionEmpleo
2,FullStack Developer Jr,Bosch,México,NaN,2021-09-30,Company Description Do you want benefici...,http://www.opcionempleo.com.mx/jobad/mxafc9a3d...,None,None,Company Description Do you want beneficial te...,None,None,None,None,OpcionEmpleo
3,Vendedores de piso de Temporada,H&M,"Zapopan, Jal.",NaN,2021-09-30,Descripción del empleo VENDEDOR DE PISO ...,http://www.opcionempleo.com.mx/jobad/mxb111512...,Zapopan,Jalisco,Descripción del empleo VENDEDOR DE PISO ¿Est...,None,None,None,None,OpcionEmpleo
4,Ayudante General,PPG,"Reynosa, Tamps.",NaN,2021-09-30,"Como Ayudante General, serás responsable d...",http://www.opcionempleo.com.mx/jobad/mxe62d685...,Reynosa,Tamaulipas,"Como Ayudante General, serás responsable...",None,None,None,None,OpcionEmpleo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,Retail Customer Advisor - SIS Monterrey,Ermenegildo Zegna,Nuevo Leon,NaN,2020-09-30,ASESOR DE VENTAS – ZEGNA SIS MONTERREY E...,http://www.opcionempleo.com.mx/jobad/mx1812aa5...,None,None,ASESOR DE VENTAS – ZEGNA SIS MONTERREY ...,None,None,None,None,OpcionEmpleo
1136,Devops,TRS Staffing Solutions,México,NaN,2020-09-30,You are subscribed to our push notificatio...,http://www.opcionempleo.com.mx/jobad/mxa1497c2...,None,None,You are subscribed to our push notificat...,None,None,None,None,OpcionEmpleo
1137,AZURE Data Engineer,TRS Staffing Solutions,México,NaN,2020-09-30,You are subscribed to our push notificatio...,http://www.opcionempleo.com.mx/jobad/mx51055fb...,None,None,You are subscribed to our push notificat...,None,None,None,None,OpcionEmpleo
1138,Field Specialist II - Production Operations,Baker Hughes,Veracruz,NaN,2020-09-30,Field Specialist-Productions Operations ...,http://www.opcionempleo.com.mx/jobad/mxdc3aa4b...,None,Veracruz,Field Specialist-Productions Operations ...,None,None,None,None,OpcionEmpleo
